# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 10:15AM,242414,19,ADV80004353,"AdvaGen Pharma, Ltd",Released
1,Jul 21 2022 10:13AM,242413,20,8303563,"Exact-Rx, Inc.",Released
2,Jul 21 2022 10:13AM,242413,20,8303564,"Exact-Rx, Inc.",Released
3,Jul 21 2022 10:13AM,242413,20,8303565,"Exact-Rx, Inc.",Released
4,Jul 21 2022 10:07AM,242405,10,0085893672,ISDIN Corporation,Released
5,Jul 21 2022 10:07AM,242405,10,0085893737,ISDIN Corporation,Released
6,Jul 21 2022 10:07AM,242405,10,0085893739,ISDIN Corporation,Released
7,Jul 21 2022 10:07AM,242405,10,0085893736,ISDIN Corporation,Released
8,Jul 21 2022 10:07AM,242405,10,0085893743,ISDIN Corporation,Released
9,Jul 21 2022 10:07AM,242405,10,0085893751,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,242410,Released,1
36,242411,Released,4
37,242412,Released,2
38,242413,Released,3
39,242414,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242410,NaN,NaN,1.0
242411,NaN,NaN,4.0
242412,NaN,NaN,2.0
242413,NaN,NaN,3.0
242414,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242305,0.0,0.0,1.0
242306,0.0,0.0,2.0
242308,0.0,0.0,1.0
242319,0.0,0.0,1.0
242325,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242305,0,0,1
242306,0,0,2
242308,0,0,1
242319,0,0,1
242325,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242305,0,0,1
1,242306,0,0,2
2,242308,0,0,1
3,242319,0,0,1
4,242325,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242305,,,1
1,242306,,,2
2,242308,,,1
3,242319,,,1
4,242325,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 10:15AM,242414,19,"AdvaGen Pharma, Ltd"
1,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc."
4,Jul 21 2022 10:07AM,242405,10,ISDIN Corporation
16,Jul 21 2022 10:04AM,242412,19,"GUSA Granules USA, Inc."
18,Jul 21 2022 10:03AM,242411,16,Sartorius Lab Products and Service
22,Jul 21 2022 9:56AM,242410,16,Sartorius Stedim Filters Inc.
23,Jul 21 2022 9:53AM,242409,10,Bio-PRF
24,Jul 21 2022 9:49AM,242408,16,Sartorius Stedim Filters Inc.
25,Jul 21 2022 9:41AM,242404,16,"SHL Pharma, LLC"
26,Jul 21 2022 9:24AM,242401,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 21 2022 10:15AM,242414,19,"AdvaGen Pharma, Ltd",,,1
1,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc.",,,3
2,Jul 21 2022 10:07AM,242405,10,ISDIN Corporation,,,12
3,Jul 21 2022 10:04AM,242412,19,"GUSA Granules USA, Inc.",,,2
4,Jul 21 2022 10:03AM,242411,16,Sartorius Lab Products and Service,,,4
5,Jul 21 2022 9:56AM,242410,16,Sartorius Stedim Filters Inc.,,,1
6,Jul 21 2022 9:53AM,242409,10,Bio-PRF,,,1
7,Jul 21 2022 9:49AM,242408,16,Sartorius Stedim Filters Inc.,,,1
8,Jul 21 2022 9:41AM,242404,16,"SHL Pharma, LLC",,,1
9,Jul 21 2022 9:24AM,242401,20,Alumier Labs Inc.,,1,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 10:15AM,242414,19,"AdvaGen Pharma, Ltd",1,,
1,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc.",3,,
2,Jul 21 2022 10:07AM,242405,10,ISDIN Corporation,12,,
3,Jul 21 2022 10:04AM,242412,19,"GUSA Granules USA, Inc.",2,,
4,Jul 21 2022 10:03AM,242411,16,Sartorius Lab Products and Service,4,,
5,Jul 21 2022 9:56AM,242410,16,Sartorius Stedim Filters Inc.,1,,
6,Jul 21 2022 9:53AM,242409,10,Bio-PRF,1,,
7,Jul 21 2022 9:49AM,242408,16,Sartorius Stedim Filters Inc.,1,,
8,Jul 21 2022 9:41AM,242404,16,"SHL Pharma, LLC",1,,
9,Jul 21 2022 9:24AM,242401,20,Alumier Labs Inc.,5,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 10:15AM,242414,19,"AdvaGen Pharma, Ltd",1,,
1,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc.",3,,
2,Jul 21 2022 10:07AM,242405,10,ISDIN Corporation,12,,
3,Jul 21 2022 10:04AM,242412,19,"GUSA Granules USA, Inc.",2,,
4,Jul 21 2022 10:03AM,242411,16,Sartorius Lab Products and Service,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 10:15AM,242414,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc.",3.0,NaN,NaN
2,Jul 21 2022 10:07AM,242405,10,ISDIN Corporation,12.0,NaN,NaN
3,Jul 21 2022 10:04AM,242412,19,"GUSA Granules USA, Inc.",2.0,NaN,NaN
4,Jul 21 2022 10:03AM,242411,16,Sartorius Lab Products and Service,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 10:15AM,242414,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc.",3.0,0.0,0.0
2,Jul 21 2022 10:07AM,242405,10,ISDIN Corporation,12.0,0.0,0.0
3,Jul 21 2022 10:04AM,242412,19,"GUSA Granules USA, Inc.",2.0,0.0,0.0
4,Jul 21 2022 10:03AM,242411,16,Sartorius Lab Products and Service,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3635592,137.0,17.0,0.0
12,242336,1.0,0.0,0.0
16,1212032,8.0,0.0,0.0
19,1454252,19.0,19.0,23.0
20,484814,8.0,1.0,0.0
21,1696490,7.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3635592,137.0,17.0,0.0
1,12,242336,1.0,0.0,0.0
2,16,1212032,8.0,0.0,0.0
3,19,1454252,19.0,19.0,23.0
4,20,484814,8.0,1.0,0.0
5,21,1696490,7.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,137.0,17.0,0.0
1,12,1.0,0.0,0.0
2,16,8.0,0.0,0.0
3,19,19.0,19.0,23.0
4,20,8.0,1.0,0.0
5,21,7.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,137.0
1,12,Released,1.0
2,16,Released,8.0
3,19,Released,19.0
4,20,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,23.0,0.0,0.0
Executing,17.0,0.0,0.0,19.0,1.0,0.0
Released,137.0,1.0,8.0,19.0,8.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,17.0,0.0,0.0,19.0,1.0,0.0
2,Released,137.0,1.0,8.0,19.0,8.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,17.0,0.0,0.0,19.0,1.0,0.0
2,Released,137.0,1.0,8.0,19.0,8.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()